<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Sentiment/merge_clarifai_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Crea il csv contenente il merge tra 
# -csv contenente tutte le recensioni del ristorante
# -csv contenente gli user presi random analizzati da Clarifai

#da cambiare:
#nome_ristorante
#users_restaurant_id (link condivisibile del csv con tutte le review)
#users_clarifai_id (link condivisibile del csv random di clarifai)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import json
import pandas as pd
import csv

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

nome_ristorante = 'TupeloHoney'
nome_ristorante_staccato = 'Tupelo Honey'

# DOWNLOAD Restaurant.csv FROM DRIVE
# users_restaurant_id = id del file csv del ristorante (si trova in data > Scraping > Reviews_Relevance) > (nomeRistorante)_relevance_desc.csv)
#File dove ci sono id e recensione
users_restaurant_id = '1aOTHWSNdkjzW7EyyaomdXxgnzjXdqN7S'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': users_restaurant_id})
download.GetContentFile('%s' % nome_ristorante_staccato + '_relevance_desc.csv')

#leggo il file csv del ristorante
restaurant = pd.read_csv('%s' % nome_ristorante_staccato + '_relevance_desc.csv')


#file con gli user che sono stati analizzati da Clarifai
#users_clarifai_id = id del file csv di clarifai (si trova in data > face_data > Clarifai > nomeRistorante > (nomeRistorante)_10%Random_UsersData.csv)
users_clarifai_id = '1lZM22vKH4DRjxgG0PIbuos4MbaF86MqU'  # FILE ID, got on google drive with condivision link
download1 = drive.CreateFile({'id': users_clarifai_id})
download1.GetContentFile('%s' % nome_ristorante + '_10%Random_UsersData.csv')

#leggo il file csv di clarifai
clarifai = pd.read_csv('%s' % nome_ristorante + '_10%Random_UsersData.csv')

merged = clarifai.merge(restaurant,how = 'left',left_on='id', right_on='user_id')

#file contenente informazioni di demographics e review per utente
df_senza_duplicati = merged.drop_duplicates(subset=['id'], keep='first')

review_random = df_senza_duplicati.drop(columns=['id','age', 'gender','ethnicity'])


#csv contenente user_id e review
review_random.to_csv('%s' % nome_ristorante + '_ReviewRandom.csv')


folder_id = '1qd07afmJktKju4tsxYfanJWEafV3JSEb'    #cartella dove metto il csv
file_metadata = {'title': '%s' % nome_ristorante + '_Random_Review.csv',"parents":[{"id": folder_id,"kind":"drive#childList"}]}
folder = drive.CreateFile(file_metadata)
folder.SetContentFile('%s' % nome_ristorante + '_Random_Review.csv') #The contents of the file
folder.Upload()